In [1]:
import Extractor
import pandas as pd
import sqlite3

conn = sqlite3.connect('Humor.db')
cur = conn.cursor()
df = pd.read_sql('SELECT head FROM head ORDER BY wdate DESC',conn)
df.drop_duplicates(keep='first', inplace=True)
ext = Extractor.Ext(df)
df = ext.cleaning()

new_words = ext.search_dict(sorted(ext.extract_nouns().items(),key=lambda _:_[1], reverse=True))
sent = ext.extract_sent(new_words)

# 변수 생성
statistic = ext.extract_statistic_value(sent)
r_rat = ext.extract_r_rat(sent,statistic)
statistic = ext.combine_var(statistic, r_rat)
conn = sqlite3.connect('Humor_var.db')
statistic.to_sql('var', conn)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 153629 from 127907 sents. mem=0.497 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=556166, mem=0.731 Gb
[Noun Extractor] batch prediction was completed for 40185 words
[Noun Extractor] checked compounds. discovered 27173 compounds
[Noun Extractor] postprocessing detaching_features : 4608 -> 4576
[Noun Extractor] postprocessing ignore_features : 4576 -> 4467
[Noun Extractor] postprocessing ignore_NJ : 4467 -> 4430
[Noun Extractor] 4430 nouns (27173 compounds) with min frequency=12
[Noun Extractor] flushing was done. mem=0.806 Gb                    
[Noun Extractor] 61.27 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 156261 from 127907 sents. mem=0.846 Gb                    
[Noun Extractor] complete eojeol counter -> l

all accessor variety was computed # words = 16336
training was done. used memory 1.048 Gb1.048 Gb
all cohesion probabilities was computed. # words = 144
all branching entropies was computed # words = 24359
all accessor variety was computed # words = 24359
training was done. used memory 1.048 Gb1.048 Gb
all cohesion probabilities was computed. # words = 157
all branching entropies was computed # words = 25297
all accessor variety was computed # words = 25297
training was done. used memory 1.048 Gb1.048 Gb
all cohesion probabilities was computed. # words = 162
all branching entropies was computed # words = 25586
all accessor variety was computed # words = 25586
training was done. used memory 1.048 Gb1.048 Gb
all cohesion probabilities was computed. # words = 165
all branching entropies was computed # words = 25784
all accessor variety was computed # words = 25784
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 166
all branching entropies

all accessor variety was computed # words = 38939
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 382
all branching entropies was computed # words = 39014
all accessor variety was computed # words = 39014
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 387
all branching entropies was computed # words = 39056
all accessor variety was computed # words = 39056
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 390
all branching entropies was computed # words = 39094
all accessor variety was computed # words = 39094
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 393
all branching entropies was computed # words = 39124
all accessor variety was computed # words = 39124
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 394
all branching entropies

all accessor variety was computed # words = 44591
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 546
all branching entropies was computed # words = 44683
all accessor variety was computed # words = 44683
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 551
all branching entropies was computed # words = 44706
all accessor variety was computed # words = 44706
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 551
all branching entropies was computed # words = 45024
all accessor variety was computed # words = 45024
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 552
all branching entropies was computed # words = 45102
all accessor variety was computed # words = 45102
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 555
all branching entropies

all accessor variety was computed # words = 48828
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 710
all branching entropies was computed # words = 49277
all accessor variety was computed # words = 49277
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 711
all branching entropies was computed # words = 49312
all accessor variety was computed # words = 49312
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 713
all branching entropies was computed # words = 49320
all accessor variety was computed # words = 49320
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 715
all branching entropies was computed # words = 49332
all accessor variety was computed # words = 49332
training was done. used memory 1.052 Gb1.052 Gb
all cohesion probabilities was computed. # words = 715
all branching entropies

...

[Noun Extractor] batch prediction was completed for 207 words
[Noun Extractor] checked compounds. discovered 2 compounds
[Noun Extractor] postprocessing detaching_features : 53 -> 53
[Noun Extractor] postprocessing ignore_features : 53 -> 48
[Noun Extractor] postprocessing ignore_NJ : 48 -> 48
[Noun Extractor] 48 nouns (2 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 41.95 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 480 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=662, mem=0.961 Gb
[Noun Extractor] batch prediction was completed for 179 words
[Noun Extractor] checked compounds. discovered 0 compounds
[Noun Extractor] postprocessing detaching_features : 32 -> 32
[Noun Extractor] postprocessing ignore_features : 32 -> 29
[Noun Extractor] postprocessing ignore_NJ : 29 -> 29
[Noun Ex

[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 676 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=983, mem=0.961 Gb
[Noun Extractor] batch prediction was completed for 232 words
[Noun Extractor] checked compounds. discovered 4 compounds
[Noun Extractor] postprocessing detaching_features : 41 -> 41
[Noun Extractor] postprocessing ignore_features : 41 -> 37
[Noun Extractor] postprocessing ignore_NJ : 37 -> 37
[Noun Extractor] 37 nouns (4 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 37.13 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 606 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=957, mem=0.961 Gb
[Noun Extractor] batch prediction was completed for 222 words
[Noun Extracto

[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 13.52 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 559 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=778, mem=0.961 Gb
[Noun Extractor] batch prediction was completed for 193 words
[Noun Extractor] checked compounds. discovered 2 compounds
[Noun Extractor] postprocessing detaching_features : 35 -> 35
[Noun Extractor] postprocessing ignore_features : 35 -> 33
[Noun Extractor] postprocessing ignore_NJ : 33 -> 33
[Noun Extractor] 33 nouns (2 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 32.90 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 865 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has bee

[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 44.35 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 887 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=1287, mem=0.961 Gb
[Noun Extractor] batch prediction was completed for 303 words
[Noun Extractor] checked compounds. discovered 7 compounds
[Noun Extractor] postprocessing detaching_features : 68 -> 67
[Noun Extractor] postprocessing ignore_features : 67 -> 61
[Noun Extractor] postprocessing ignore_NJ : 61 -> 61
[Noun Extractor] 61 nouns (7 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=0.961 Gb                    
[Noun Extractor] 33.10 % eojeols are covered
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 472 from 1 sents. mem=0.961 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has be

ValueError: Table 'var' already exists.

In [14]:
statistic

,cohesion_forward,cohesion_backward,left_branching_entropy,right_branching_entropy,left_accessor_variety,right_accessor_variety,leftside_frequency,rightside_frequency,right_post_postion_ratio,right_whitespace_ratio
확진자,0.984480,0.260976,4.114816,3.894225,190.0,211.0,4563.0,274.0,0.172913,0.756520
알아보자,0.997411,0.000000,1.744232,4.692180,47.0,445.0,11400.0,0.0,0.007019,0.964815
문재앙,0.966864,0.000000,4.684202,4.508473,240.0,243.0,2797.0,0.0,0.209660,0.722004
얘들아,0.997093,0.000000,4.213548,4.150799,179.0,253.0,2911.0,0.0,0.004823,0.972442
스압,0.873124,0.518752,4.241932,4.977995,334.0,460.0,7852.0,5353.0,0.004904,0.636822
...,...,...,...,...,...,...,...,...,...,...
유세윤,0.069765,0.000000,2.789972,2.308084,24.0,15.0,130.0,0.0,0.184615,0.746154
흑형,0.211278,0.000000,3.638187,3.122885,76.0,42.0,547.0,0.0,0.338208,0.484461
상남자,0.208081,0.000000,3.937743,3.812566,87.0,68.0,573.0,0.0,0.190559,0.625874
넣고,0.993103,0.000000,2.117934,2.607551,17.0,25.0,144.0,0.0,0.041667,0.819444
